In [2]:
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import warnings
warnings.filterwarnings("ignore")
import time
from pinecone import Pinecone
import tqdm
from dotenv import load_dotenv
import os

In [3]:

import requests

response = requests.get("https://www.gutenberg.org/cache/epub/64317/pg64317.txt")

In [7]:
book = response.text
book = book[5:]
len(book)

296853

In [8]:
file_path = "./book.txt"

with open(file_path, "w", encoding="utf-8") as f:
    f.write(book)

In [9]:
with open(file_path, "r",  encoding="utf-8") as f:
    text = f.read()

text = text.replace('\t', ' ')

In [10]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", "\t"], chunk_size=1200, chunk_overlap=100)

In [11]:
documents = text_splitter.create_documents([text])
num_documents = len(documents)

print (f"Now our book is split up into {num_documents} documents")

Now our book is split up into 270 documents


In [12]:

len(documents)

270

In [17]:
embeddings = OllamaEmbeddings(model="llama2:latest", show_progress=True, num_gpu=1)

In [18]:
import datetime
texts = [doc.page_content for doc in documents]

In [19]:
texts[0]

'Project Gutenberg eBook of The Great Gatsby\n\n    \n\nThis ebook is for the use of anyone anywhere in the United States and\n\nmost other parts of the world at no cost and with almost no restrictions\n\nwhatsoever. You may copy it, give it away or re-use it under the terms\n\nof the Project Gutenberg License included with this ebook or online\n\nat www.gutenberg.org. If you are not located in the United States,\n\nyou will have to check the laws of the country where you are located\n\nbefore using this eBook.\n\n\n\nTitle: The Great Gatsby\n\n\n\nAuthor: F. Scott Fitzgerald\n\n\n\nRelease date: January 17, 2021 [eBook #64317]\n\n\n\nLanguage: English\n\n\n\nCredits: Produced by Alex Cabal for the Standard Ebooks project, based on a transcription produced for Project Gutenberg Australia.\n\n\n\n\n\n*** START OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***\n\n\n\n\n\n\n\n\n\n                           The Great Gatsby\n\n                                  by\n\n                     

In [20]:
vectors = []
doc_nt = []
for doc,i in zip(texts, range(len(texts))):
    emb = embeddings.embed_documents(texts= [doc])
    if emb[0] != None:
        vectors.append(emb[0])
    else:
        print(f"document {i}, not treated first 20 chars {doc[:20]}")
        doc_nt.append(i)

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.20s/it]


In [21]:
doc_nt

[]

In [22]:
len(vectors[-1])

4096

In [23]:
len(vectors)

270

In [24]:
env_path = os.path.join( ".env")
load_dotenv(env_path)

True

In [25]:
PINECONE_API_KEY= os.getenv("PINECONE_API_KEY")

In [32]:
pinecone = Pinecone(api_key=PINECONE_API_KEY)

In [33]:
INDEX_NAME = "rag"

In [34]:
index = pinecone.Index(INDEX_NAME)

In [35]:
prepped = []

for i , em, text in zip(range(len(vectors)), vectors, texts):
    v = { "id" : f"vec{i}",
        "values" : em,
        "metadata" : {"text": text}}
    prepped.append(v)
    if len(prepped) >= 25:
        index.upsert(prepped)
        prepped = []
index.upsert(prepped)    

{'upserted_count': 20}

In [36]:
def get_embeddings(articles, embeddings):
   return embeddings.embed_documents(texts= articles)

In [37]:
query = "who is the Great Gastby?"

embed = get_embeddings([query], embeddings)


OllamaEmbeddings: 100%|██████████| 1/1 [00:09<00:00,  9.59s/it]


In [38]:
embed

[[2.2404680252075195,
  -0.040541838854551315,
  1.324207067489624,
  0.31291764974594116,
  -2.2001099586486816,
  1.2731740474700928,
  1.1627492904663086,
  -0.6060039401054382,
  -0.6760268807411194,
  -0.36386168003082275,
  1.1122076511383057,
  -1.0694880485534668,
  -2.1121373176574707,
  2.3817718029022217,
  -0.11286754906177521,
  -0.8849610090255737,
  -1.2218700647354126,
  -0.00015440289280377328,
  0.7411199808120728,
  -2.7964413166046143,
  0.7579671740531921,
  -1.8343610763549805,
  0.8731238842010498,
  -2.003049373626709,
  0.33917832374572754,
  -1.5026246309280396,
  0.978951632976532,
  -1.0732148885726929,
  0.6667384505271912,
  -1.198336124420166,
  1.5286204814910889,
  -1.472557783126831,
  -0.5950385928153992,
  4.986698150634766,
  2.394962787628174,
  -5.109634876251221,
  -0.7257654070854187,
  0.9254480004310608,
  0.09300629794597626,
  -1.155437707901001,
  -2.4117448329925537,
  -4.352016448974609,
  0.9741517305374146,
  -1.9598970413208008,
  -0.5

In [39]:
res = index.query(vector=embed, top_k=3, include_metadata=True)


In [40]:
type(res)

pinecone.core.openapi.data.model.query_response.QueryResponse

In [41]:
response = res.to_dict()

In [42]:
response.keys()

dict_keys(['matches', 'namespace', 'usage'])

In [43]:
response['usage']

{'read_units': 6}

In [44]:
response['matches'][0]

{'id': 'vec237',
 'score': 0.585009098,
 'values': [],
 'metadata': {'text': 'He was so hard up he had to keep on wearing his uniform because he\n\ncouldn’t buy some regular clothes. First time I saw him was when he\n\ncame into Winebrenner’s poolroom at Forty-third Street and asked for a\n\njob. He hadn’t eat anything for a couple of days. ‘Come on have some\n\nlunch with me,’ I said. He ate more than four dollars’ worth of food\n\nin half an hour.”\n\n\n\n“Did you start him in business?” I inquired.\n\n\n\n“Start him! I made him.”\n\n\n\n“Oh.”\n\n\n\n“I raised him up out of nothing, right out of the gutter. I saw right\n\naway he was a fine-appearing, gentlemanly young man, and when he told\n\nme he was at Oggsford I knew I could use him good. I got him to join\n\nthe American Legion and he used to stand high there. Right off he did\n\nsome work for a client of mine up to Albany. We were so thick like\n\nthat in everything”—he held up two bulbous fingers—“always together.”\n\n\n\nI w

In [45]:
for r in response['matches']:
    print(f" Score {r.get('score')}, text {r.get('metadata').get('text')}")

 Score 0.585009098, text He was so hard up he had to keep on wearing his uniform because he

couldn’t buy some regular clothes. First time I saw him was when he

came into Winebrenner’s poolroom at Forty-third Street and asked for a

job. He hadn’t eat anything for a couple of days. ‘Come on have some

lunch with me,’ I said. He ate more than four dollars’ worth of food

in half an hour.”



“Did you start him in business?” I inquired.



“Start him! I made him.”



“Oh.”



“I raised him up out of nothing, right out of the gutter. I saw right

away he was a fine-appearing, gentlemanly young man, and when he told

me he was at Oggsford I knew I could use him good. I got him to join

the American Legion and he used to stand high there. Right off he did

some work for a client of mine up to Albany. We were so thick like

that in everything”—he held up two bulbous fingers—“always together.”



I wondered if this partnership had included the World’s Series

transaction in 1919.



“Now he’